# Deep Learning India - Assignment 2 : CIFAR-10 using Densenet

##### **Problem** : Achieve an accuracy beyond 85% for a Image classification using CIFAR-10 dataset based on [Densenet](https://arxiv.org/pdf/1608.06993.pdf) paper

##### Sample CIFAR-10 dataset

![CIFAR-10 Dataset](https://github.com/praveenraghuvanshi1512/Learning/blob/DeepLearningIndia_VV/Technical/AIML/DeepLearningIndia/Assignment_2/images/cifar2.jpg?raw=true)

### Steps ###
1. Initial Setup and configuration
2. Load CIFAR-10 dataset
3. Prepare Data
4. Create model pipeline
5. Compile Model
6. Train/Fit Model
7. Evaluate Model

### Step 1: Initial Setup and Configuration

In [ ]:
# STEP 1: Initial Setup and Configuration

# 1.1 Set theano as backend to keras
from keras import backend as k
import os
import importlib

def set_keras_backend(backend):
    if k.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(k)
        assert k.backend() == backend

set_keras_backend("theano")

# 1.2 Enable intellisense in jupyter notebook
%config IPCompleter.greedy=True

#### Utility Functions
##### Display Images

In [ ]:
# Utility functions

from matplotlib import pyplot
from PIL import Image

# Display images
def show_images(x):
    pyplot.figure(1)
    k = 0
    for i in range(0, 4):
        for j in range(0, 4):
            pyplot.subplot2grid((4, 4), (i, j))
            pyplot.imshow(Image.fromarray(x[k]))
            k = k + 1
    # Show the plot
    pyplot.show()

##### Plot losses

In [ ]:
# Plot Losses

def plotLosses(history):  
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

### Step 2: Load CIFAR-10 dataset

In [ ]:
# Step 2: Load CIFAR-10 dataset
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data();

##### Examining Dataset

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

- There are 50000 training and 10000 testing images
- CIFAR-10 dataset has color images of shape(width, height, channels) : 32 x 32 x 3
- No of channels for a colored image : 3(Red + Green + Blue)
- No of classes : 10 - Type of objects in images such as airplane, bird, truck etc.

In [ ]:
# Display first 16 images

show_images(X_test[:16])

### Step 3: Prepare data

In [ ]:
# STEP 3: Prepare Data

from keras.utils import np_utils
import numpy as np

# Transform images from (32, 32, 3) to 3072-dimensional vector(32*32*3)
X_train = np.reshape(X_train,(50000,3072))
X_test = np.reshape(X_test,(10000,3072))

# Converting unsigned integers to 32-bit float precision
# so that it can be better utilized during normalization such as diving by mac pixel value of 255
X_train = X_train.astype(np.float32)
X_test =  X_test.astype(np.float32)

# Normalization of pixel values (to [0-1] range)
X_train /= 255
X_test /= 255

# Convert class vectors to binary class matrices
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

print(y_train[0])

##### Examining Dataset

In [ ]:
print(X_train.shape, "X_train")
print(X_test.shape, "X_test")
print(y_train.shape,"y_train")
print(y_test.shape, "y_train")

In [ ]:
print('X_train[0] :\n', X_train[0])
print('\n')
print('y_train[0] :\n', y_train[0, 0][0])

### Step 4: Create Model Pipeline - Multi Layer perceptron(MLP)

In [ ]:
# STEP 4: Create Model pipeline

from keras.models import Sequential
from keras.layers import Dense, Activation

# Input shape is a tensor of image.
# CIFAR-10 has images of dimension (witdth * height * channel) as 32 x 32 x 3
input_dim = 32 * 32 * 3 # 3072
print(input_dim, 'input dimension')

# Learned parameter calculation :
# https://www.youtube.com/watch?v=gmBfb6LNnZs
# No of parameters = (Previous layer channels * Current layer filters * kernel) + bias(current filter count)

# Create model - Input Layer
# Learned Parameter: 0 . No parameters to be learned in input layer
model = Sequential()

# Now we'll add hidden layers

# Dense layer - Its a fully connected layer where all neurons from previous are connected to all neurons in dense layer
# Activation - These functions allows non-linear transformations to be performed on a given input which is a complex process
# Learned Parameter: (3072 * 256) + 256 = 786688
model.add(Dense(256, activation='relu', input_dim=input_dim))

# Learned Parameter: (256 * 256) + 256 = 65792
model.add(Dense(256, activation='relu'))

# Learned Parameter: (256 * 10) + 10 = 2570
model.add(Dense(num_classes, activation='softmax'))

model.summary()

### Step 5: Compile Model - Configures model for training

In [ ]:
# STEP 5: Compile Model : Configures model for training

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Step 6: Train/Fit Model
Save training loss and accuracy to history object for later visualization

In [ ]:
# STEP 6: Train/Fit Model

history = model.fit(X_train, y_train,
              batch_size=32,
              epochs=10,
              validation_data=(X_test, y_test))

##### Plot Losses

In [ ]:
# Plot losses
plotLosses(history)

### Step 7: Evaluate

In [ ]:
# Evaluate model
score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)

In [ ]:
print(model.metrics_names)
print(score)

### Observations

Image classification using Multilevel Perceptron(MLP) on CIFAR-10 dataset.

- Neural network with 3 Dense layer
- Very large no of parameters (855,050)
- Time consuming : Didn't complete training
- No Shareable parameters
- Inefficient network

### References

- [CIFAR-10 Demo](http://home.mit.bme.hu/~hadhazi/Oktatas/NN18/dem3/html_demo/CIFAR-10Demo.html)
- [Learnable Parametersin a Convolutional Neural Network(CNN) explained](https://www.youtube.com/watch?v=gmBfb6LNnZs)
- [How to calculate the number of parameters in the CNN?](https://medium.com/@iamvarman/how-to-calculate-the-number-of-parameters-in-the-cnn-5bd55364d7ca#targetText=To%20calculate%20it%2C%20we%20have,3%E2%80%931))%20%3D%2048.)
- [Understanding and Calculating the number of Parameters in Convolution Neural Networks (CNNs)](https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d)
- [Calculate model parameters in Convolutional Neural Networks (CIFAR-10)](https://stats.stackexchange.com/questions/336908/calculate-model-parameters-in-convolutional-neural-networks-cifar-10)